In [1]:
import sys
import os

sys.path.append(os.path.abspath("../.."))

In [2]:
from pythonbacktest.datafeed import CSVDataFeed
from pythonbacktest.backtestengine import BasicBackTestEngine
from pythonbacktest.strategy import import_strategy
from pythonbacktest.broker import BackTestBroker

In [3]:
TEST_DATA_PATH = os.path.abspath("../testdata/ACME")
INITIAL_BUDGET = 100000

In [4]:
csv_data_feed = CSVDataFeed()
csv_data_feed.load_data(TEST_DATA_PATH)

In [5]:
broker = BackTestBroker(INITIAL_BUDGET)

In [6]:
strategy_module = import_strategy(
    "basicSMAstrategy", 
    os.path.abspath("basicsmastrategy.py"))

strategy = strategy_module.BasicSMAStrategy()

In [7]:
back_test_engine = BasicBackTestEngine(csv_data_feed, strategy, broker)
back_test_engine.start()

<bound method StaticValue.result of <pythonbacktest.indicator.staticvalue.StaticValue object at 0x107003650>>
<bound method StaticValue.result of <pythonbacktest.indicator.staticvalue.StaticValue object at 0x107003650>>
<bound method StaticValue.result of <pythonbacktest.indicator.staticvalue.StaticValue object at 0x107003650>>
<bound method StaticValue.result of <pythonbacktest.indicator.staticvalue.StaticValue object at 0x107003650>>
<bound method StaticValue.result of <pythonbacktest.indicator.staticvalue.StaticValue object at 0x107003650>>
<bound method StaticValue.result of <pythonbacktest.indicator.staticvalue.StaticValue object at 0x107003650>>
<bound method StaticValue.result of <pythonbacktest.indicator.staticvalue.StaticValue object at 0x107003650>>
<bound method StaticValue.result of <pythonbacktest.indicator.staticvalue.StaticValue object at 0x107003650>>
<bound method StaticValue.result of <pythonbacktest.indicator.staticvalue.StaticValue object at 0x107003650>>
<bound met